### homework 1: COVID-19 Cases Prediction (regression)

### download data
you can download data from [kaggle](https://www.kaggle.com/c/ml2021spring-hw1/data), and upload data manually to the workspace

In [ ]:
tr_path = 'covid.train.csv'  # path to training data
tt_path = 'covid.test.csv'   # path to testing data

### import some packages

In [ ]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# for data preprocess
import numpy as np
import csv
import os

# for plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

myseed = 42069  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(myseed)

### some utilities
you do not need to modify this part

In [ ]:
def get_device():
  ''' get device (if GPU is available, use GPU) '''
  return 'cuda' if torch.cuda.is_available() else 'cpu'

def plot_learning_curve(loss_record, title=''):
  ''' plot learning curve of your DNN (train & dev loss) '''
  total_steps = len(loss_record['train'])
  x_1 = range(total_steps)
  x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
  figure(figsize=(6, 4))
  plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
  plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
  plt.ylim(0.0, 5.)
  plt.xlabel('training steps')
  plt.ylabel('MSE loss')
  plt.title('learning curve of {}'.format(title))
  plt.legend()
  plt.show()


def plot_pred(dv_set, model, device, lim=35., preds=None, targets=None):
  ''' plot prediction of your DNN '''
  if preds is None or targets is None:
    model.eval()
    preds, targets = [], []
    for x, y in dv_set:
      x, y = x.to(device), y.to(device)
      with torch.no_grad():
        pred = model(x)
        preds.append(pred.detach().cpu())
        targets.append(y.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    targets = torch.cat(targets, dim=0).numpy()

  figure(figsize=(5, 5))
  plt.scatter(targets, preds, c='r', alpha=0.5)
  plt.plot([-0.2, lim], [-0.2, lim], c='b')
  plt.xlim(-0.2, lim)
  plt.ylim(-0.2, lim)
  plt.xlabel('ground truth value')
  plt.ylabel('predicted value')
  plt.title('ground truth v.s. prediction')
  plt.show()

### preprocess
we have three kinds of datasets:
* train: for training
* dev: for validation
* test: for testing (w/o target value)

### feature selection


#### feature filter

  

In [ ]:
import pandas as pd
data_tr = pd.read_csv(tr_path) # read training data
data_tt = pd.read_csv(tt_path) # read testing data

data_tr.drop('id', axis=1, inplace=True) # drop 'id' column
data_tt.drop('id', axis=1, inplace=True)

data_corr = data_tr.iloc[:, 40:].corr()  # compute pairwise correlation of columns
target_col = data_corr['tested_positive.2']
feature = target_col[target_col > 0.8]

feature_cols = feature.index.tolist()  # get column name
feature_cols.pop()                     # remove test_positive 

cols = list(data_tr.columns)  # list of feature names

# get the index of the feature column
# which can be used as feats + feats_selected for feature values
feats_selected = [cols.index(col) for col in feature_cols]
feats_selected

### Dataset
the COVID19Dataset below does:
* read `.csv` files
* extract features
* split `covid.train.csv` into train/dev sets
* normalize features

finishing TODO below might make you pass medium baseline

In [ ]:
class COVID19Dataset(Dataset):
  ''' Dataset for loading and preprocessing the COVID19 dataset '''

  def __init__(self,
               path,
               mode='train',
               target_only=False):
    self.mode = mode

    # read data into numpy arrays
    with open(path, 'r') as fp:
      data = list(csv.reader(fp))
      data = np.array(data[1:])[:, 1:].astype(float)

    if not target_only:
      feats = list(range(93))
    else:
      # TODO: using 40 states & 2 tested_positive features (indices = 57 & 75)
      # feats = list(range(40)) + [57, 75]
      feats = list(range(40)) + feats_selected

    if mode == 'test':
      # testing data
      # data: 893 x 93 (40 states + day 1 (18) + day 2 (18) + day 3 (17))
      data = data[:, feats]
      self.data = torch.FloatTensor(data)
    else:
      # training data (train/dev sets)
      # data: 2700 x 94 (40 states + day 1 (18) + day 2 (18) + day 3 (18))
      target = data[:, -1]
      data = data[:, feats]

      # splitting training data into train & dev sets
      if mode == 'train':
        indices = [i for i in range(len(data)) if i % 10 != 0]
      elif mode == 'dev':
        indices = [i for i in range(len(data)) if i % 10 == 0]

      # convert data into PyTorch tensors
      self.data = torch.FloatTensor(data[indices])
      self.target = torch.FloatTensor(target[indices])

    # normalize features (you may remove this part to see what will happen)
    self.data[:, 40:] = \
        (self.data[:, 40:] - self.data[:, 40:].mean(dim=0, keepdim=True)) \
        / self.data[:, 40:].std(dim=0, keepdim=True)

    self.dim = self.data.shape[1]

    print('finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
          .format(mode, len(self.data), self.dim))

  def __getitem__(self, index):
    if self.mode in ['train', 'dev']:
      return self.data[index], self.target[index]
    else:
      return self.data[index]

  def __len__(self):
    return len(self.data)

### DataLoader
a DataLoader loads data from a given Dataset into batches.

In [ ]:
def prep_dataloader(path, mode, batch_size, n_jobs=0, target_only=False):
  ''' generates a dataset, then is put into a dataloader '''
  dataset = COVID19Dataset(path, mode=mode, target_only=target_only)
  dataloader = DataLoader(dataset, batch_size,
                          shuffle=(mode == 'train'), drop_last=False,
                          num_workers=n_jobs, pin_memory=True)
  return dataloader

### deep neural network
`NeuralNet` is an nn.Module designed for regression. 

the DNN consists of 2 fully-connected layers with `ReLU` activation. 

this module also included a function `cal_loss` for calculating loss.

In [ ]:
class NeuralNet(nn.Module):
  ''' a simple fully-connected deep neural network '''

  def __init__(self, input_dim):
    super().__init__()

    # define your neural network here
    # TODO: how to modify this model to achieve better performance?
    self.net = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 1)
    )

    # mean squared error loss
    self.criterion = nn.MSELoss(reduction='mean')

  def forward(self, x):
    ''' given input of size (batch_size x input_dim), compute output of the network '''
    return self.net(x).squeeze(1)

  def cal_loss(self, pred, target):
    ''' calculate loss '''
    # TODO: you may implement L1/L2 regularization here
    return self.criterion(pred, target)

### training

In [ ]:
def train(tr_set, dv_set, model, config, device):
  ''' DNN training '''

  n_epochs = config['n_epochs']  # maximum number of epochs

  # setup optimizer
  optimizer = getattr(torch.optim, config['optimizer'])(
      model.parameters(), **config['optim_hparas'])

  min_mse = 1000.
  loss_record = {'train': [], 'dev': []}      # for recording training loss
  early_stop_cnt = 0
  epoch = 0
  while epoch < n_epochs:
    model.train()                           # set model to training mode
    for x, y in tr_set:                     # iterate through the dataloader
      optimizer.zero_grad()                 # set gradient to zero
      # move data to device (cpu/cuda)
      x, y = x.to(device), y.to(device)
      # forward pass (compute output)
      pred = model(x)
      mse_loss = model.cal_loss(pred, y)    # compute loss
      mse_loss.backward()                   # compute gradient (backpropagation)
      optimizer.step()                      # update model with optimizer
      loss_record['train'].append(mse_loss.detach().cpu().item())

    # after each epoch, test your model on the validation (development) set.
    dev_mse = dev(dv_set, model, device)
    if dev_mse < min_mse:
      # save model if your model improved
      min_mse = dev_mse
      print('saving model (epoch = {:4d}, loss = {:.4f})'
            .format(epoch + 1, min_mse))
      # save model to specified path
      torch.save(model.state_dict(), config['save_path'])
      early_stop_cnt = 0
    else:
      early_stop_cnt += 1

    epoch += 1
    loss_record['dev'].append(dev_mse)
    if early_stop_cnt > config['early_stop']:
      # stop training if your model stops improving for "config['early_stop']" epochs.
      break

  print('finished training after {} epochs'.format(epoch))
  return min_mse, loss_record

### validation

In [ ]:
def dev(dv_set, model, device):
  model.eval()                                # set model to evalutation mode
  total_loss = 0
  for x, y in dv_set:                         # iterate through the dataloader
    # move data to device (cpu/cuda)
    x, y = x.to(device), y.to(device)
    with torch.no_grad():                   # disable gradient calculation
      # forward pass (compute output)
      pred = model(x)
      mse_loss = model.cal_loss(pred, y)    # compute loss
    total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
  # compute averaged loss
  total_loss = total_loss / len(dv_set.dataset)

  return total_loss

### testing

In [ ]:
def test(tt_set, model, device):
  model.eval()                                # set model to evalutation mode
  preds = []
  for x in tt_set:                            # iterate through the dataloader
    # move data to device (cpu/cuda)
    x = x.to(device)
    with torch.no_grad():                   # disable gradient calculation
      # forward pass (compute output)
      pred = model(x)
      preds.append(pred.detach().cpu())     # collect prediction
  # concatenate all predictions and convert to a numpy array
  preds = torch.cat(preds, dim=0).numpy()
  return preds

### setup hyper-parameters
config contains hyper-parameters for training and the path to save your model

In [ ]:
# get the current available device ('cpu' or 'cuda')
device = get_device()
# the trained model will be saved to ./models/
os.makedirs('models', exist_ok=True)
# TODO: using 40 states & 2 tested_positive features
target_only = True

# TODO: how to tune these hyper-parameters to improve your model's performance?
config = {
  'n_epochs': 3000,                # maximum number of epochs
  'batch_size': 270,               # mini-batch size for dataloader
  # optimization algorithm (optimizer in torch.optim)
  'optimizer': 'SGD',
  'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
    'lr': 0.001,                   # learning rate of SGD
    'momentum': 0.9                # momentum for SGD
  },
  # early stopping epochs (the number epochs since your model's last improvement)
  'early_stop': 200,
  'save_path': 'models/model.pth'  # your model will be saved here
}

### load data and model

In [ ]:
tr_set = prep_dataloader(tr_path, 'train', config['batch_size'], target_only=target_only)
dv_set = prep_dataloader(tr_path, 'dev', config['batch_size'], target_only=target_only)
tt_set = prep_dataloader(tt_path, 'test', config['batch_size'], target_only=target_only)

In [ ]:
model = NeuralNet(tr_set.dataset.dim).to(device)  # construct model and move to device

### start training!

In [ ]:
model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)

In [ ]:
plot_learning_curve(model_loss_record, title='deep model')

In [ ]:
del model
model = NeuralNet(tr_set.dataset.dim).to(device)
# load your best model
ckpt = torch.load(config['save_path'], map_location='cpu', weights_only=True)
model.load_state_dict(ckpt)
plot_pred(dv_set, model, device)  # show prediction on the validation set

### testing
the predictions of your model on testing set will be stored at `pred.csv`

In [ ]:
def save_pred(preds, file):
  ''' save predictions to specified file '''
  print('saving results to {}'.format(file))
  with open(file, 'w') as fp:
    writer = csv.writer(fp)
    writer.writerow(['id', 'tested_positive'])
    for i, p in enumerate(preds):
      writer.writerow([i, p])


preds = test(tt_set, model, device)  # predict COVID-19 cases with your model
save_pred(preds, 'pred.csv')         # save prediction file to pred.csv

### download
run this block to download the `pred.csv` by clicking

In [ ]:
from IPython.display import FileLink
FileLink(r'pred.csv')

### baseline
| Model   | Score   |
|---------|---------|
| Simple  | 2.04826 |
| Medium  | 1.36937 |
| Strong  | 0.89266 |

### simple baseline
run sample code, submit `pred.csv` to [kaggle](https://www.kaggle.com/c/ml2021spring-hw1/data).
get a **Public Score: 1.35374** and a **Private score: 1.43738**. surpassing the simple baseline score of **2.04826**

### medium baseline
* feature selection: 40 states + 2 `tested_positive` (`TODO` in dataset)

submit `pred.csv` to [kaggle](https://www.kaggle.com/c/ml2021spring-hw1/data).
get a **Public Score: 1.05926** and a **Private score: 1.06473**. surpassing the simple baseline score of **1.36937**

### strong Baseline
* feature selection (what other features are useful?)
* DNN architecture (layers? dimension? activation function?)
* training (mini-batch? optimizer? learning rate?)
* L2 regularization
* there are some mistakes in the sample code, can you find them?

#### feature selection

In [ ]:
import pandas as pd
data_tr = pd.read_csv(tr_path) # read training data
data_tt = pd.read_csv(tt_path) # read testing data

In [ ]:
data_tr.head(5) # view the first 5 rows

In [ ]:
data_tt.head(5) # view the first 5 rows

In [ ]:
data_tr.drop('id', axis=1, inplace=True) # drop 'id' column
data_tt.drop('id', axis=1, inplace=True)

In [ ]:
data_tr.info() #check the data types and sizes of each column


In [ ]:
data_tr.iloc[:, 40:].describe() # check the data distribution

In [ ]:
# compare the test set distribution with the training set; 
# the differences in feature distributions are not significant
data_tt.iloc[:, 40:].describe() # check the data distribution

In [ ]:
# manually check the correlation between cli features and the target variable
plt.scatter(data_tr.loc[:, 'cli'], data_tr.loc[:, 'tested_positive.2'])


In [ ]:
plt.scatter(data_tr.loc[:, 'ili'], data_tr.loc[:, 'tested_positive.2'])

In [ ]:
# cli and ili are similar, so we can use just one of these features
plt.scatter(data_tr.loc[:, 'cli'], data_tr.loc[:, 'ili'])

In [ ]:
# the correlation between day 1 and day 3 target values is linear
plt.scatter(data_tr.loc[:,  'tested_positive'], data_tr.loc[:, 'tested_positive.2']) 

In [ ]:
# the correlation between day 2 and day 3 target values is linear
plt.scatter(data_tr.loc[:,  'tested_positive.1'], data_tr.loc[:, 'tested_positive.2'])

In [ ]:
# manual analysis above is too tedious
# let's use the corr method for automated analysis instead
data_tr.iloc[:, 40:].corr() # compute pairwise correlation of columns

In [ ]:
# select the last column of the correlation matrix to 
# view the correlation of other features with the target value.
data_corr = data_tr.iloc[:, 40:].corr()
target_col = data_corr['tested_positive.2']
target_col

In [ ]:
# select features with a correlation greater than 0.8
# the threshold can be adjusted as needed.
feature = target_col[target_col > 0.8]
feature

In [ ]:
feature_cols = feature.index.tolist()  # get column name
feature_cols.pop()                     # remove test_positive 
feature_cols


In [ ]:
cols = list(data_tr.columns)  # list of feature names

# get the index of the feature column
# which can be used as feats + feats_selected for feature values
feats_selected = [cols.index(col) for col in feature_cols]
feats_selected